# Projeto

## Data Preparation

- Eliminar Colunas IDentificadoras e tornar ID como PK

- Eliminar Erros de Input
    - Format "Age" & Check SSN pattern
    - Negative Numeric Variables: 'Age', 'Num_Bank_Accounts', 'NumofLoan'
    - Age Restriction (GREATER than 0 && LESS than 150)
<br><br>
- Eliminar 9 Records de 'Monthly_Balance' igual a -3.33333E+26

- Trabalhar com 'Credit_History_Age' como variavel numerica 

- Variable Enconding of 'Age' and 'Credit_History_Age'

In [1]:
import pandas as pd
import re
import numpy as np
import dslabs_functions as dslab
from math import pi, sin, cos

In [2]:
file_tag = 'credit_score'
filename = 'class_credit_score.csv'
original_dataset: pd.DataFrame = pd.read_csv(filename, index_col='ID', sep=',', decimal='.', na_values='')

# Drop IDentifying Columns
dataset_cleaned = original_dataset.drop(columns=['Customer_ID', 'Name', 'SSN'], errors='ignore')

# Erase Input Errors
## Check and Clean 'SSN' & 'Age' Column
pattern = r'^\d{3}-\d{2}-\d{4}$'  # Replace this with your SSN regex pattern
#dataset_cleaned['SSN'] = dataset_cleaned['SSN'].apply(lambda x: x if re.match(pattern, str(x)) else np.nan)
dataset_cleaned['Age'] = dataset_cleaned['Age'].astype(str).str.replace('_', '', regex=False)

## Transfrom 'Age' from SIMBOLIC to NUMERIC
dataset_cleaned['Age'] = pd.to_numeric(dataset_cleaned['Age'], errors='coerce')

## Invalid Numeric Variables 'Age', 'Num_Bank_Accounts', 'NumofLoan' Column
dataset_cleaned['Age'] = dataset_cleaned['Age'].apply(lambda x: np.nan if x == -500 or x >= 150 else x)
dataset_cleaned['Num_Bank_Accounts'] = dataset_cleaned['Num_Bank_Accounts'].apply(lambda x: x if x >= 0 else np.nan)
dataset_cleaned['NumofLoan'] = dataset_cleaned['NumofLoan'].apply(lambda x: x if x >= 0 else np.nan)

# Monthly_Balance 
dataset_cleaned['MonthlyBalance'] = dataset_cleaned['MonthlyBalance'].apply(lambda x: x if x >= 0 else np.nan)

## Transform 'Credit_History_Age' from SIMBOLIC to NUMERIC
hist_age = dataset_cleaned["Credit_History_Age"]
hist_vals = []

# Turn the months part using cyclic 
for val in hist_age:
    matches = re.findall(r'(\d+) Years and (\d+) Months', str(val))
    if not matches:
        hist_vals.append(np.nan)
    else:
        hist_vals.append(float(matches[0][0]) + (float(matches[0][1]) / 12))

dataset_cleaned["Credit_History_Age"] = hist_vals
print(hist_vals)
### Drop 'Credit_History_Age' as OBJECT 
object_columns = dataset_cleaned.select_dtypes(include=['object']).columns.tolist()
if "CreditHistoryAge" in object_columns:
    dataset_cleaned.drop(columns=["CreditHistoryAge"], inplace=True)

# Get Variables Type Right (Object to Category)
variables_types: dict[str, list] = dslab.get_variable_types(dataset_cleaned)
numeric: list[str] = variables_types["numeric"]
symbolic: list[str] = variables_types["symbolic"]
binary: list[str] = variables_types["binary"]

print(variables_types)

# Object to Category (same as symbolic in Pandas)
#dataset_cleaned[symbolic] = dataset_cleaned[symbolic].apply(lambda x: x.astype("category"))
print(dataset_cleaned.dtypes)

[22.083333333333332, nan, 22.25, 22.333333333333332, 22.416666666666668, 22.5, 22.583333333333332, nan, 26.583333333333332, 26.666666666666668, 26.75, 26.833333333333332, 26.916666666666668, 27.0, 27.083333333333332, 27.166666666666668, 17.75, 17.833333333333332, 17.916666666666668, nan, 18.083333333333332, 18.166666666666668, 18.25, 18.333333333333332, 17.25, 17.333333333333332, 17.416666666666668, 17.5, 17.583333333333332, 17.666666666666668, 17.75, 17.833333333333332, 30.666666666666668, 30.75, 30.833333333333332, 30.916666666666668, 31.0, 31.083333333333332, 31.166666666666668, 31.25, nan, 32.0, nan, 32.166666666666664, 32.25, nan, 32.416666666666664, 32.5, 30.583333333333332, 30.666666666666668, 30.75, 30.833333333333332, 30.916666666666668, 31.0, 31.083333333333332, 31.166666666666668, nan, 14.666666666666666, 14.75, 14.833333333333334, 14.916666666666666, 15.0, 15.083333333333334, 15.166666666666666, 21.333333333333332, 21.416666666666668, 21.5, 21.583333333333332, 21.6666666666

### Variable Enconding

- *'Credit_History_Age' feito na 1ª célula*

1. Order Enconding Binary *(order is irrelevant)*
    - 'Credit_Score' <br><br>

2. Order Enconding *(order is relevant)*
    - 'CreditMix' [Bad, Standard, Good] (3)<br><br>
    - 'Payment_of_Min_Amount' ['No', 'NM', 'Yes'] (3) <br><br>
    - 'Payment_Behaviour' ['Low_spent_Small_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Large_value_payments', 
        'High_spent_Small_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Large_value_payments'] (6) (2 different columns???)<br><br>
    - 'Occupation' <br><br>

3. Cyclic Variables
    - 'Month' ordenar primeiro [January, February, March, ...] (usado no inicio no credit_history_age tambem)<br><br> 

4. Other technic
    - 'Type_of_Loan' ['Student Loan', 'Payday Loan', 'Not Specified', 'Home Equity Loan', 'Auto Loan', 
        'Personal Loan', 'Debt Consolidation Loan', 'Credit-Builder Loan', 'Mortgage Loan']

In [3]:
# Collect Individual values for each symbolic var, in order to encode the rest
dataset_cleaned

for v in variables_types["symbolic"]:
    print(v, dataset_cleaned[v].unique())

Month ['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August']
Occupation ['Scientist' nan 'Teacher' 'Engineer' 'Entrepreneur' 'Developer' 'Lawyer'
 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant' 'Musician'
 'Mechanic' 'Writer' 'Architect']
Type_of_Loan ['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan'
 'Credit-Builder Loan' 'Auto Loan, Auto Loan, and Not Specified' ...
 'Home Equity Loan, Auto Loan, Auto Loan, and Auto Loan'
 'Payday Loan, Student Loan, Mortgage Loan, and Not Specified'
 'Personal Loan, Auto Loan, Mortgage Loan, Student Loan, and Student Loan']
CreditMix [nan 'Good' 'Standard' 'Bad']
Payment_of_Min_Amount ['No' 'NM' 'Yes']
Payment_Behaviour ['High_spent_Small_value_payments' 'Low_spent_Large_value_payments'
 'Low_spent_Medium_value_payments' 'Low_spent_Small_value_payments'
 'High_spent_Medium_value_payments' nan 'High_spent_Large_value_payments']


In [4]:
# Enconding 'Data_Score' Binary - Ordinal Enconding (easiest one)
variables_types

yes_no: dict[str, int] = {"no": 0, "No": 0, "yes": 1, "Yes": 1}
credit_score_values: dict[str, int] = {"Poor": 0, "Good": 1}

encoding_binary: dict[str, dict[str, int]] = {
    "Credit_Score": credit_score_values,
}

data_encoded_binary: pd.DataFrame = dataset_cleaned.replace(encoding_binary, inplace=False)

# Enconding 'Credit_Mix' Non-Binary - Ordinal Enconding
credit_mix_values: dict[str, int] = {"Bad": 0, "Standard": 1, "Good": 2}

# Enconding 'Payment_of_Min_Amount' Non-Binary - Ordinal Enconding
payment_of_min_amount_values: dict[str, int] = {"No": 0, "NM": 1, "Yes": 2}

# Enconding 'Payment_Behaviour' Non-Binary - Ordinal Enconding
payment_behaviour_values: dict[str, int] = {"Low_spent_Small_value_payments": 0, "Low_spent_Medium_value_payments": 1, "Low_spent_Large_value_payments": 2,
                                             "High_spent_Small_value_payments": 3, "High_spent_Medium_value_payments": 4, "High_spent_Large_value_payments": 5}

encoding_non_binary: dict[str, dict[str, int]] = {
    "CreditMix": credit_mix_values,
    "Payment_of_Min_Amount": payment_of_min_amount_values,
    "Payment_Behaviour": payment_behaviour_values,
}

data_encoded_non_binary: pd.DataFrame = data_encoded_binary.replace(encoding_non_binary, inplace=False)

# Enconding 'Month' - Cyclic variables
# Na coluna 'Month' so ha info ate ao mês 8 (Agosto) - devemos reduzir dict?
month_values: dict[str, float] = {
    "January": 0,
    "February": 1/6 * pi,
    "March": 2/6 * pi,
    "April": 3/6 * pi,
    "May": 4/6 * pi,
    "June": 5/6 * pi,
    "July": pi,
    "August": -5/6 * pi,
    "September": -4/6 * pi,
    "October": -3/6 * pi,
    "November": -2/6 * pi,
    "December": -1/6 * pi,
}

enconding_cyclic: dict[str, dict] = {
    "Month": month_values,
}

data_encoded_cyclic = data_encoded_non_binary.replace(enconding_cyclic)
data_encoded_cyclic.head(3)

## QUAL E A IDEIA DISTO???? (se manter nao mudar nome variavel)
# data_encoded_cyclic: pd.DataFrame | None = dslab.encode_cyclic_variables_2(data_encoded_cyclic, ["Month"])
# if data_encoded_cyclic is not None:
#     data_encoded_cyclic.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,NumofLoan,Type_of_Loan,...,CreditMix,OutstandingDebt,CreditUtilizationRatio,Credit_History_Age,Payment_of_Min_Amount,TotalEMIpermonth,Amountinvestedmonthly,Payment_Behaviour,MonthlyBalance,Credit_Score
ID,,,,,,,,,,,,,,,,,,,,,
0x1602,0.000000,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,NaN,809.98,26.822620,22.083333,0,49.574949,80.415295,3.0,312.494089,1
0x1603,0.523599,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2.0,809.98,31.944960,NaN,0,49.574949,118.280222,2.0,284.629163,1
0x1604,1.047198,NaN,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2.0,809.98,28.609352,22.250000,0,49.574949,81.699521,1.0,331.209863,1


*Separar em colunas e usar contador*

In [5]:
# # Enconding 'Type_of_Loan' - Dummification
# type_of_loans = data_encoded_cyclic["Type_of_Loan"]
# unique_loans_list = []
# credit_score_index = data_encoded_cyclic.columns.get_loc("Credit_Score")

# for loan in type_of_loans:
    
#     matches = re.findall(r'([\w\s-]+)(?:, | and |$)', str(loan))
#     m_filtered = [m if "and" not in m else m[4:] for m in matches]
#     m_filtered_series = pd.Series(m_filtered)

#     unique_loans_list.extend(m_filtered)

# # drop nan in list of unique values (don't use it for enconding)
# unique_loans_list = list(set(unique_loans_list))
# unique_loans_list = [item for item in unique_loans_list if item != 'nan']
# #['Student Loan', 'Payday Loan', 'Not Specified', 'Home Equity Loan', 'Auto Loan', 
# # 'Personal Loan', 'Debt Consolidation Loan', 'Credit-Builder Loan', 'Mortgage Loan']

# # Create new Columns for Dummification
# for loan_type in unique_loans_list:
#     data_encoded_cyclic.insert(credit_score_index, loan_type, 0)

# # Check which Loans there is in the original column and update Loan types
# for index, row in data_encoded_cyclic.iterrows():
#     types = re.findall(r'([\w\s-]+)(?:, | and |$)', str(row["Type_of_Loan"]))
#     types_filtered = [t if "and" not in t else t[4:] for t in types]
    
#     for loan_type in types_filtered:
#         if loan_type in unique_loans_list:
#             data_encoded_cyclic.at[index, loan_type] += 1

# ## If all elements are 0, change them for NaN??

# # Drop original "Type_of_Loan"
# data_encoded_cyclic.drop("Type_of_Loan", axis=1, inplace=True)

# #data_encoded_cyclic.rename(columns={'Not Specified': 'Not Specified Loan'}, inplace=True)

# data_encoded_cyclic.head(9)

*Separar em colunas e usar contador + incrementar tendo em conta nivel de risco (0 a 9)*

In [6]:
# Enconding 'Type_of_Loan' - Dummification
type_of_loans = data_encoded_cyclic["Type_of_Loan"]
unique_loans_list = []
credit_score_index = data_encoded_cyclic.columns.get_loc("Credit_Score")

for loan in type_of_loans:
    
    matches = re.findall(r'([\w\s-]+)(?:, | and |$)', str(loan))
    m_filtered = [m if "and" not in m else m[4:] for m in matches]
    m_filtered_series = pd.Series(m_filtered)

    unique_loans_list.extend(m_filtered)

# drop nan in list of unique values (don't use it for enconding)
unique_loans_list = list(set(unique_loans_list))
unique_loans_list = [item for item in unique_loans_list if item != 'nan']
#['Student Loan', 'Payday Loan', 'Not Specified', 'Home Equity Loan', 'Auto Loan', 
# 'Personal Loan', 'Debt Consolidation Loan', 'Credit-Builder Loan', 'Mortgage Loan']

# Cresce conforme nivel de risco
type_of_loans_values = {
    "Not Specified": 8,
    "Debt Consolidation Loan": 7,
    "Home Equity Loan": 6,
    "Mortgage Loan": 5,
    "Personal Loan": 4,
    "Auto Loan": 3,
    "Student Loan": 2,
    "Payday Loan": 1,
    "Credit-Builder Loan": 0,
}

# Create new Columns for Dummification
for loan_type in unique_loans_list:
    data_encoded_cyclic.insert(credit_score_index, loan_type, 0)

# Check which Loans there is in the original column and update Loan types
for index, row in data_encoded_cyclic.iterrows():
    types = re.findall(r'([\w\s-]+)(?:, | and |$)', str(row["Type_of_Loan"]))
    types_filtered = [t if "and" not in t else t[4:] for t in types]
    
    for loan_type in types_filtered:
        if loan_type in unique_loans_list:
             # Increment by the value from type_of_loans_values
            data_encoded_cyclic.at[index, loan_type] += type_of_loans_values.get(loan_type, 0)

## If all elements are 0, change them for NaN??

# Drop original "Type_of_Loan"
data_encoded_cyclic.drop("Type_of_Loan", axis=1, inplace=True)

data_encoded_cyclic.rename(columns={'Not Specified': 'Not Specified Loan'}, inplace=True)

data_encoded_cyclic.head(9)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,NumofLoan,Delay_from_due_date,...,Not Specified Loan,Auto Loan,Mortgage Loan,Credit-Builder Loan,Debt Consolidation Loan,Student Loan,Personal Loan,Home Equity Loan,Payday Loan,Credit_Score
ID,,,,,,,,,,,,,,,,,,,,,
0x1602,0.000000,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,3,...,0,3,0,0,0,0,4,6,0,1
0x1603,0.523599,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,-1,...,0,3,0,0,0,0,4,6,0,1
0x1604,1.047198,NaN,Scientist,19114.12,NaN,3.0,4,3,4.0,3,...,0,3,0,0,0,0,4,6,0,1
0x1605,1.570796,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,5,...,0,3,0,0,0,0,4,6,0,1
0x1606,2.094395,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,6,...,0,3,0,0,0,0,4,6,0,1
0x1607,2.617994,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,8,...,0,3,0,0,0,0,4,6,0,1
0x1608,3.141593,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,3,...,0,3,0,0,0,0,4,6,0,1
0x1609,-2.617994,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,3,...,0,3,0,0,0,0,4,6,0,1
0x160e,0.000000,28.0,NaN,34847.84,3037.986667,2.0,4,6,1.0,3,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# Enconding 'Occupation' - Ordinal Enconding Hierarchical Values
######### Using Hierarchical Values
occupation_values = {
    "Scientist": 2,
    "Engineer": 1,
    "Developer": 0,
    "Doctor": 3,
    "Architect": 10,
    "Writer": 11,
    "Musician": 12,
    "Entrepreneur": 4,
    "Manager": 5,
    "Accountant": 6,
    "Media_Manager": 13,
    "Lawyer": 9,
    "Mechanic": 7,
    "Teacher": 8,
    "Journalist": 14,
}

enconding_occupation: dict[str, dict[str, int]] = {
    "Occupation": occupation_values,
}

data_encoded_hierarchical = pd.DataFrame = data_encoded_cyclic.replace(enconding_occupation, inplace=False)
######### Using Hierarchical Values

unique_occupation = data_encoded_hierarchical["Occupation"].dropna().unique()

# drop nan in list of unique values (don't use it for enconding)
unique_occupation_list = list(set(unique_occupation))
unique_occupation_list = [item for item in unique_occupation_list if item != 'nan']

######### Using Dummification
# vars_to_dummify: list[str] = ["Occupation"]
# all_data_encoded: pd.DataFrame = dslab.dummify_numeric(data_encoded_cyclic, vars_to_dummify)
# all_data_encoded.drop("Occupation_nan", axis=1, inplace=True)
######### Using Dummification

# Eliminate repeated 'Credit_Score' by index
#all_data_encoded = data_encoded_hierarchical.iloc[:, [i for i in range(data_encoded_hierarchical.shape[1]) if i != 30]]
#all_data_encoded['Occupation']

### Separar em Treino e Teste + Obter Metricas

In [8]:
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import savefig, show, figure

data_encoded_hierarchical.to_csv('data_preparation_csvs/enconding_csvs/data_encoded.csv', index=True)

# falar sobre a escolha do K

In [9]:
# def evaluate_approach(
#     train: pd.DataFrame, test: pd.DataFrame, target: str = "class", metric: str = "accuracy"
# ) -> dict[str, list]:
#     trnY = train.pop(target).values
#     trnX: np.ndarray = train.values
#     tstY = test.pop(target).values
#     tstX: np.ndarray = test.values
#     eval: dict[str, list] = {}

#     eval_NB: dict[str, float] = dslab.naive_Bayes_study(trnX, trnY, tstX, tstY, metric=metric)
#     eval_KNN: dict[str, float] = dslab.knn_study(trnX, trnY, tstX, tstY, metric=metric)
#     if eval_NB != {} and eval_KNN != {}:
#         for met in dslab.CLASS_EVAL_METRICS:
#             eval[met] = [eval_NB[met], eval_KNN[met]]
#     return eval

In [10]:
# train_data, test_data = train_test_split(all_data_encoded, test_size=0.2, random_state=42)

# train_data.to_csv('data_preparation_csvs/data_train_encoded.csv', index=True)
# test_data.to_csv('data_preparation_csvs/data_test_encoded.csv', index=True)

# Evaluate
# target = "Credit_Score"

# figure()
# eval: dict[str, list] = evaluate_approach(train_data, test_data, target=target, metric="recall")
# dslab.plot_multibar_chart(
#     ["NB", "KNN"], eval, title=f"{file_tag} evaluation", percentage=True
# )
# savefig(f"data_preparation_images/data_encoded_eval.png")
# show()